In [7]:
import threading
import time

import pandas as pd

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.trade_app import TradeApp
import lib.helper as helper

In [8]:
app = TradeApp()


In [9]:
"""Function that run the entire trading app """
app = TradeApp()
app.connect(host='127.0.0.1', port=7497,
                clientId=23)  # port 4002 for ib gateway paper trading/7497 for TWS paper trading
con_thread = threading.Thread(target=app.run, daemon=True)
con_thread.start()

tickers = ["FB", "AMZN", "INTC", "MSFT", "AAPL", "GOOG", "CSCO", "CMCSA", "ADBE", "NVDA",
               "NFLX", "PYPL", "AMGN", "AVGO", "TXN", "CHTR", "QCOM", "GILD", "FISV", "BKNG",
               "INTU", "ADP", "CME", "TMUS", "MU"]

capital = 300_000



In [10]:
app.reqPositions()
time.sleep(2)
pos_df = app.pos_df
pos_df.drop_duplicates(inplace=True, ignore_index=True)  # position callback tends to give duplicate values
app.reqOpenOrders()
time.sleep(2)

ERROR -1 504 Not connected
ERROR -1 504 Not connected


In [11]:
for ticker in tickers:
    print("starting passthrough for.....",ticker)
    app.req_hist_data(tickers.index(ticker), helper.contract(ticker),'1 M', '15 mins')
    time.sleep(5)

ERROR 0 504 Not connected


starting passthrough for..... FB


ERROR 1 504 Not connected


starting passthrough for..... AMZN


ERROR 2 504 Not connected


...

ERROR 6 504 Not connected


starting passthrough for..... CSCO


ERROR 7 504 Not connected


starting passthrough for..... CMCSA


KeyboardInterrupt: 

In [13]:
app.hist_data

{}